In [1]:
print("Sa")

Sa


In [2]:
# Cell 1 — Imports & basic utilities (kept stdlib-only to run anywhere)
from __future__ import annotations

import json
import math
import os
from dataclasses import dataclass, field, asdict
from datetime import datetime, timedelta, date
from typing import Any, Dict, List, Optional, Tuple
from urllib.parse import urlencode
import urllib.request

print("Ready. Stdlib-only setup loaded.")


Ready. Stdlib-only setup loaded.


In [29]:
# Cell 2 — Weather schema (lightweight dataclass matching your hint) and validators

# Cell 2a — Patch: timezone-aware ISO now (silences UTC deprecation warning)
from datetime import datetime, timezone

def _iso_now_utc():
    # Produce RFC-3339/ISO string with 'Z' suffix for UTC
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat().replace("+00:00", "Z")

print("Patched _iso_now_utc() to be timezone-aware.")


@dataclass
class WeatherSchema:
    # Core meta
    lat: Optional[float] = None
    lon: Optional[float] = None
    elevation: Optional[float] = None
    tz: Optional[str] = None
    run_at: Optional[str] = None  # ISO-8601 UTC
    
    # Daily arrays
    time: List[str] = field(default_factory=list)
    tmin_c: List[Optional[float]] = field(default_factory=list)
    tmax_c: List[Optional[float]] = field(default_factory=list)
    rain_mm: List[Optional[float]] = field(default_factory=list)
    et0_mm: List[Optional[float]] = field(default_factory=list)
    
    # Daily rollups (single values over the window; computed if possible)
    wind_speed_10m_ms: Optional[float] = None  # mean of hourly over window
    rh_mean_pct: Optional[float] = None        # mean of hourly over window
    shortwave_radiation_mj_m2: Optional[float] = None  # sum of daily shortwave
    
    # Hourly arrays
    time_hourly: List[str] = field(default_factory=list)
    temp_2m_c: List[Optional[float]] = field(default_factory=list)
    precip_mm: List[Optional[float]] = field(default_factory=list)
    et0_hourly_mm: List[Optional[float]] = field(default_factory=list)
    wind_speed_hourly_10m_ms: List[Optional[float]] = field(default_factory=list)
    rh_2m_pct: List[Optional[float]] = field(default_factory=list)

    # Internal: source metadata (handy for citations/traceability)
    source: Dict[str, Any] = field(default_factory=dict)

    def to_dict(self) -> Dict[str, Any]:
        return asdict(self)


def _validate_lat_lon(lat: Any, lon: Any) -> Tuple[float, float]:
    try:
        lat = float(lat)
        lon = float(lon)
    except Exception:
        raise ValueError("lat/lon must be convertible to float")
    if not (-90.0 <= lat <= 90.0):
        raise ValueError("lat must be between -90 and 90")
    if not (-180.0 <= lon <= 180.0):
        raise ValueError("lon must be between -180 and 180")
    return lat, lon


def _parse_date_yyyy_mm_dd(s: Optional[str]) -> Optional[date]:
    if not s:
        return None
    return datetime.strptime(s, "%Y-%m-%d").date()


print("Schema & validators ready.")


Patched _iso_now_utc() to be timezone-aware.
Schema & validators ready.


In [30]:
# Cell 3 — Open-Meteo request builder, safe fetch with offline fallback

OPEN_METEO_BASE = "https://api.open-meteo.com/v1/forecast"

def _build_open_meteo_url(lat: float, lon: float, start: date, end: date) -> str:
    # We request both daily and hourly variables.
    daily_vars = [
        "temperature_2m_max",
        "temperature_2m_min",
        "precipitation_sum",
        "et0_fao_evapotranspiration",
        "shortwave_radiation_sum",
        "wind_speed_10m_max",
    ]
    hourly_vars = [
        "temperature_2m",
        "precipitation",
        "et0_fao_evapotranspiration",
        "wind_speed_10m",
        "relative_humidity_2m",
        "shortwave_radiation",
    ]
    params = {
        "latitude": lat,
        "longitude": lon,
        "timezone": "auto",
        "start_date": start.isoformat(),
        "end_date": end.isoformat(),
        "daily": ",".join(daily_vars),
        "hourly": ",".join(hourly_vars),
    }
    return OPEN_METEO_BASE + "?" + urlencode(params)


def _http_get_json(url: str) -> Dict[str, Any]:
    try:
        with urllib.request.urlopen(url, timeout=30) as resp:
            data = json.loads(resp.read().decode("utf-8"))
            return data
    except Exception as e:
        # Offline or network error — return a small realistic stub so the notebook still runs.
        today = datetime.utcnow().date()
        stub = {
            "latitude": 12.97,
            "longitude": 77.59,
            "elevation": 920.0,
            "timezone": "Asia/Kolkata",
            "timezone_abbreviation": "IST",
            "generationtime_ms": 5.2,
            "utc_offset_seconds": 19800,
            "daily": {
                "time": [today.isoformat()],
                "temperature_2m_max": [30.5],
                "temperature_2m_min": [22.3],
                "precipitation_sum": [4.2],
                "et0_fao_evapotranspiration": [3.1],
                "shortwave_radiation_sum": [18.7],
                "wind_speed_10m_max": [6.2],
            },
            "hourly": {
                "time": [f"{today.isoformat()}T{str(h).zfill(2)}:00" for h in range(24)],
                "temperature_2m": [22.0 + 0.4 * h if 0 <= h <= 12 else 26.8 - 0.3 * (h - 12) for h in range(24)],
                "precipitation": [0.0] * 20 + [0.4, 0.8, 0.2, 0.0],
                "et0_fao_evapotranspiration": [0.05] * 24,
                "wind_speed_10m": [2.0 + 0.2 * (h % 6) for h in range(24)],
                "relative_humidity_2m": [85 - 1.5 * (h % 12) for h in range(24)],
                "shortwave_radiation": [0 if h < 6 or h > 18 else 200 + 30 * (h - 12) for h in range(24)],
            },
        }
        stub["_offline_reason"] = str(e)
        return stub


print("Open-Meteo URL builder & safe fetch ready.")


Open-Meteo URL builder & safe fetch ready.


In [31]:
# Cell 4 — Core function: weather_lookup(args) returning your schema + source stamp

def _mean(values: List[Optional[float]]) -> Optional[float]:
    nums = [v for v in values if isinstance(v, (int, float)) and not math.isnan(v)]
    return sum(nums) / len(nums) if nums else None

def _sum(values: List[Optional[float]]) -> Optional[float]:
    nums = [v for v in values if isinstance(v, (int, float)) and not math.isnan(v)]
    return sum(nums) if nums else None


def weather_lookup(args: Dict[str, Any]) -> Dict[str, Any]:
    """Fetch weather from Open-Meteo and return structured dict following the provided schema.
    
    Args keys: lat (float), lon (float), start_date (YYYY-MM-DD, optional), end_date (YYYY-MM-DD, optional)
    Defaults: [today .. today+6] if dates omitted.
    Raises: ValueError for invalid/missing lat/lon.
    """
    if not isinstance(args, dict):
        raise ValueError("args must be a dict")
    if "lat" not in args or "lon" not in args:
        raise ValueError("Missing required keys: 'lat' and 'lon'")
    
    lat, lon = _validate_lat_lon(args["lat"], args["lon"])
    
    # Date window
    start = _parse_date_yyyy_mm_dd(args.get("start_date"))
    end = _parse_date_yyyy_mm_dd(args.get("end_date"))
    if start and end and end < start:
        raise ValueError("end_date cannot be before start_date")
    if start is None and end is None:
        start = datetime.utcnow().date()
        end = start + timedelta(days=6)
    elif start is None and end is not None:
        start = end - timedelta(days=6)
    elif start is not None and end is None:
        end = start + timedelta(days=6)
    
    # Build URL and fetch
    url = _build_open_meteo_url(lat, lon, start, end)
    raw = _http_get_json(url)
    
    # Parse core meta
    tz = raw.get("timezone") or raw.get("timezone_abbreviation")
    elevation = raw.get("elevation")
    hourly = raw.get("hourly", {})
    daily = raw.get("daily", {})
    
    # Daily mapping
    w = WeatherSchema()
    w.lat = raw.get("latitude", lat)
    w.lon = raw.get("longitude", lon)
    w.elevation = elevation
    w.tz = tz
    w.run_at = _iso_now_utc()
    
    w.time = daily.get("time", [])
    w.tmax_c = daily.get("temperature_2m_max", [])
    w.tmin_c = daily.get("temperature_2m_min", [])
    w.rain_mm = daily.get("precipitation_sum", [])
    w.et0_mm = daily.get("et0_fao_evapotranspiration", [])
    
    # Hourly mapping
    w.time_hourly = hourly.get("time", [])
    w.temp_2m_c = hourly.get("temperature_2m", [])
    w.precip_mm = hourly.get("precipitation", [])
    w.et0_hourly_mm = hourly.get("et0_fao_evapotranspiration", [])
    w.wind_speed_hourly_10m_ms = hourly.get("wind_speed_10m", [])
    w.rh_2m_pct = hourly.get("relative_humidity_2m", [])
    
    # Rollups over the window
    w.wind_speed_10m_ms = _mean([float(x) if x is not None else None for x in w.wind_speed_hourly_10m_ms])
    w.rh_mean_pct = _mean([float(x) if x is not None else None for x in w.rh_2m_pct])
    # For shortwave daily sum, Open-Meteo daily provides MJ/m²
    shortwave_daily = daily.get("shortwave_radiation_sum", [])
    w.shortwave_radiation_mj_m2 = _sum([float(x) if x is not None else None for x in shortwave_daily])
    
    # Source metadata (traceable URL + window)
    source_stamp = {
        "provider": "open-meteo",
        "url": url,
        "window": {"start_date": start.isoformat(), "end_date": end.isoformat()},
        "issued_utc": w.run_at,
    }
    if "_offline_reason" in raw:
        source_stamp["offline_reason"] = raw["_offline_reason"]
        source_stamp["note"] = "Stubbed data due to network restriction in this environment."
    w.source = source_stamp
    
    return w.to_dict()


print("weather_lookup(args) is defined.")


weather_lookup(args) is defined.


In [32]:
# Cell 5 — Register as a LangChain Tool (uses real Tool if available, else a tiny shim)

LC_TOOL = None
try:
    # Try modern LangChain core first
    try:
        from langchain_core.tools import Tool
    except Exception:
        # Older LangChain
        from langchain.tools import Tool  # type: ignore
    
    LC_TOOL = Tool.from_function(
        name="weather_lookup",
        func=weather_lookup,
        description=(
            "Fetches daily+hourly weather from Open-Meteo given {lat, lon, start_date?, end_date?}. "
            "Returns a dict following the project weather schema with arrays and rollups."
        ),
    )
    print("LangChain Tool created via langchain.*")
except Exception as e:
    # Minimal shim so your planner can still call .name/.description/.run
    class _ToolShim:
        def __init__(self, name, func, description):
            self.name = name
            self.description = description
            self.func = func
        def run(self, tool_input):
            return self.func(tool_input)
        @property
        def args(self):
            return {"type": "object", "properties": {"lat": {"type":"number"}, "lon":{"type":"number"}}}
    LC_TOOL = _ToolShim(
        name="weather_lookup",
        func=weather_lookup,
        description=(
            "[Shim] Fetches daily+hourly weather from Open-Meteo given {lat, lon, start_date?, end_date?}. "
            "Returns a dict following the project weather schema."
        ),
    )
    print("LangChain not found — using a lightweight Tool shim.")


LangChain Tool created via langchain.*


In [33]:
# Cell 5b — Fix registration: use StructuredTool so dict input works

LC_TOOL = None
try:
    try:
        from pydantic import BaseModel
    except Exception:
        # pydantic is a hard dependency of langchain; if somehow missing, raise a clear error
        raise ImportError("pydantic is required for StructuredTool args_schema")
    
    try:
        from langchain_core.tools import StructuredTool
    except Exception:
        # Fallback for older langchain versions
        from langchain.tools import StructuredTool  # type: ignore
    
    class WeatherArgs(BaseModel):
        lat: float
        lon: float
        start_date: Optional[str] = None  # YYYY-MM-DD
        end_date: Optional[str] = None    # YYYY-MM-DD
    
    LC_TOOL = StructuredTool.from_function(
        name="weather_lookup",
        func=weather_lookup,
        args_schema=WeatherArgs,
        description=(
            "Fetches daily+hourly weather from Open-Meteo. "
            "Args: lat (float), lon (float), start_date?, end_date? (YYYY-MM-DD). "
            "Returns a dict following the project weather schema."
        ),
    )
    print("LangChain StructuredTool created (dict input supported).")
except Exception as e:
    # Minimal shim so your planner can still call .invoke(dict)
    class _StructuredToolShim:
        def __init__(self, name, func, description):
            self.name = name
            self.description = description
            self.func = func
        def invoke(self, tool_input: Dict[str, Any]):
            if not isinstance(tool_input, dict):
                raise ValueError("tool_input must be a dict with keys lat, lon, ...")
            return self.func(tool_input)
        # Back-compat .run for code already written
        def run(self, tool_input):
            return self.invoke(tool_input if isinstance(tool_input, dict) else {})
    LC_TOOL = _StructuredToolShim(
        name="weather_lookup",
        func=weather_lookup,
        description=(
            "[Shim] Fetches daily+hourly weather from Open-Meteo. Dict input supported."
        ),
    )
    print("LangChain not available — using a StructuredTool shim.", str(e))


LangChain StructuredTool created (dict input supported).


In [34]:
# Cell 5c — StructuredTool registration using a kwargs→dict adapter

LC_TOOL = None
try:
    from pydantic import BaseModel
    try:
        from langchain_core.tools import StructuredTool
    except Exception:
        from langchain.tools import StructuredTool  # older LC

    class WeatherArgs(BaseModel):
        lat: float
        lon: float
        start_date: Optional[str] = None  # YYYY-MM-DD
        end_date: Optional[str] = None    # YYYY-MM-DD

    # Adapter: StructuredTool will call this with keyword args
    def weather_lookup_adapter(
        lat: float,
        lon: float,
        start_date: Optional[str] = None,
        end_date: Optional[str] = None,
    ):
        return weather_lookup(
            {
                "lat": lat,
                "lon": lon,
                "start_date": start_date,
                "end_date": end_date,
            }
        )

    LC_TOOL = StructuredTool.from_function(
        name="weather_lookup",
        func=weather_lookup_adapter,
        args_schema=WeatherArgs,
        description=(
            "Fetch daily+hourly weather from Open-Meteo for {lat, lon, start_date?, end_date?} "
            "and return the project weather schema dict."
        ),
    )
    print("LangChain StructuredTool registered with kwargs adapter.")
except Exception as e:
    # Minimal shim preserving .invoke(dict) and .run(dict)
    class _StructuredToolShim:
        def __init__(self, name, func, description):
            self.name = name
            self.description = description
            self.func = func
        def invoke(self, tool_input: Dict[str, Any]):
            if not isinstance(tool_input, dict):
                raise ValueError("tool_input must be a dict with keys lat, lon, ...")
            return self.func(tool_input)
        def run(self, tool_input):
            return self.invoke(tool_input if isinstance(tool_input, dict) else {})
    LC_TOOL = _StructuredToolShim(
        name="weather_lookup",
        func=weather_lookup,
        description="[Shim] Dict input supported.",
    )
    print("StructuredTool unavailable — using shim.", str(e))


LangChain StructuredTool registered with kwargs adapter.


In [38]:
# Cell 6 — Demo: Call the tool with sample coordinates using .invoke(dict)

sample_args = {
    "lat": 12.9716,
    "lon": 77.5946,
    # Optional:
    "start_date": "2025-08-15",
    "end_date": "2025-08-21",
}

# Use .invoke for StructuredTool, else fallback to direct call
demo_result = (
    LC_TOOL.invoke(sample_args)
    if hasattr(LC_TOOL, "invoke")
    else weather_lookup(sample_args)
)

# Pretty print summary
print("=== Weather Tool Demo (Bengaluru) ===")
print("Keys:", list(demo_result.keys())[:8], "...")
print("Days:", len(demo_result.get("time", [])), "| Hours:", len(demo_result.get("time_hourly", [])))
print("TZ:", demo_result.get("tz"), "| Elevation:", demo_result.get("elevation"))
print("Issued:", demo_result.get("run_at"))
print("Source:", json.dumps(demo_result.get("source", {}), ensure_ascii=False))


=== Weather Tool Demo (Bengaluru) ===
Keys: ['lat', 'lon', 'elevation', 'tz', 'run_at', 'time', 'tmin_c', 'tmax_c'] ...
Days: 7 | Hours: 168
TZ: Asia/Kolkata | Elevation: 910.0
Issued: 2025-08-15T11:48:15Z
Source: {"provider": "open-meteo", "url": "https://api.open-meteo.com/v1/forecast?latitude=12.9716&longitude=77.5946&timezone=auto&start_date=2025-08-15&end_date=2025-08-21&daily=temperature_2m_max%2Ctemperature_2m_min%2Cprecipitation_sum%2Cet0_fao_evapotranspiration%2Cshortwave_radiation_sum%2Cwind_speed_10m_max&hourly=temperature_2m%2Cprecipitation%2Cet0_fao_evapotranspiration%2Cwind_speed_10m%2Crelative_humidity_2m%2Cshortwave_radiation", "window": {"start_date": "2025-08-15", "end_date": "2025-08-21"}, "issued_utc": "2025-08-15T11:48:15Z"}


# 🛠️ Weather Tool (LangChain StructuredTool) — Usage Guide

This notebook exposes a **LangChain `StructuredTool`** named `LC_TOOL` that fetches weather from **Open-Meteo** and returns a dict matching our project’s schema. It’s designed for the **LLM-1 planner** to call directly.

---

## What the Tool Does

- **API:** Open-Meteo (no key needed)
- **Inputs:** `lat`, `lon`, optional `start_date`, `end_date` (YYYY-MM-DD)
- **Outputs:** A dict following our weather schema (daily + hourly + rollups) **plus** a `source` stamp for traceability.

---

## Contract (Do Not Break)

- **Call signature (for LangChain):**
  - Use `LC_TOOL.invoke({"lat": <float>, "lon": <float>, "start_date": "YYYY-MM-DD"?, "end_date": "YYYY-MM-DD"?})`
- **Validation:**
  - Raises `ValueError` if `lat`/`lon` missing or out of range; or if `end_date < start_date`.
- **Defaults:**
  - If dates omitted → fetches a 7-day window (`today` → `today + 6`).
- **Determinism:**
  - Results vary with time & Open-Meteo updates; `source.issued_utc` records run time.

---

## Return Schema (keys & units)

```json
{
  "lat": float|null,
  "lon": float|null,
  "elevation": float|null,
  "tz": "Area/City"|null,
  "run_at": "YYYY-MM-DDTHH:MM:SSZ"|null,

  "time": [ "YYYY-MM-DD", ... ],
  "tmin_c": [ float|null, ... ],
  "tmax_c": [ float|null, ... ],
  "rain_mm": [ float|null, ... ],
  "et0_mm": [ float|null, ... ],

  "wind_speed_10m_ms": float|null,                // mean of hourly over window
  "rh_mean_pct": float|null,                      // mean of hourly over window
  "shortwave_radiation_mj_m2": float|null,        // sum of daily MJ/m² over window

  "time_hourly": [ "YYYY-MM-DDTHH:00", ... ],
  "temp_2m_c": [ float|null, ... ],
  "precip_mm": [ float|null, ... ],
  "et0_hourly_mm": [ float|null, ... ],
  "wind_speed_hourly_10m_ms": [ float|null, ... ],
  "rh_2m_pct": [ float|null, ... ],

  "source": {
    "provider": "open-meteo",
    "url": "https://api.open-meteo.com/v1/forecast?...",
    "window": { "start_date": "YYYY-MM-DD", "end_date": "YYYY-MM-DD" },
    "issued_utc": "YYYY-MM-DDTHH:MM:SSZ",
    "offline_reason": "..."? ,   // present only if stub used
    "note": "Stubbed data..."?   // present only if stub used
  }
}
